# Don't get sideswiped by a low num_ctx value!

Try summarizing a long web page such as https://cnn.com with with llama3.2  and keep an eye on the `ollama serve` output. You may see a "truncating input prompt" warning that looks like this:

```
time=2025-01-05T17:16:18.172-08:00 level=WARN source=runner.go:129 msg="truncating input prompt" limit=2048 prompt=3940 keep=5 new=2048
```

According to https://github.com/ollama/ollama/blob/main/docs/modelfile.md, Ollama defaults the context window size to 2048. When the above warning is generated, the results returned by the model are pretty poor. It looks like the scraped webpage is being truncated before it is fed to the model.

Unfortunately, it is pretty easy to miss this warning in the `ollama serve` output. On the upside, bumping up num_ctx is easy to do – just specify the num_ctx option.

If you are using `ollama.chat()`:

```
response = ollama.chat(model=MODEL, messages=messages, options={"num_ctx": 4096})
```

See the following thread in the course Q&A: https://indeed.udemy.com/course/llm-engineering-master-ai-and-large-language-models/learn/lecture/46869605#questions/22842263

In [0]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
from openai import OpenAI

In [0]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [0]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

# The exercise using ollama.chat()

In [0]:
# A class to represent a web page

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [0]:
# The messages (system and user prompts) to send to Ollama

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [0]:
# The code to call Ollama and summarize a website
# Remove options={"num_ctx": 4096} to see the "truncating input prompt" warning

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages, options={"num_ctx": 4096})
    return response['message']['content']

In [0]:
# The code to display a website summary in Markdown

def display_summary(summary):
    display(Markdown(summary))

In [0]:
# Now summarize an actual website

url = "https://cnn.com"
summary = summarize(url)
display_summary(summary)

# The exercise using ollama_via_openai.chat.completions.create()

In [0]:
# The OpenAI API is subject to the same default 2048 num_ctx issue. Unfortunately, upping num_ctx
# isn't as straightforward with this API. See https://github.com/ollama/ollama/blob/main/docs/openai.md
# for a way to create a new model with its num_ctx bumped up. Once you have the new model you can specify
# it in the chat.completions.create() arguments.

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,       # Where you can specify your new model once you create it
    messages=messages_for(Website("https://cnn.com")),
    # max_tokens=4096  # This doesn't work
)

display_summary(response.choices[0].message.content)